In [1]:
import numpy as np
from math import pi
import pandas as pd
from scipy.spatial import distance
import pandas as pd

from qiskit.quantum_info import Kraus, SuperOp
from qiskit.providers.aer import AerSimulator
from qiskit.tools.visualization import plot_histogram

# Import from Qiskit Aer noise module
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import thermal_relaxation_error
from qiskit.providers.ibmq.job import job_monitor

from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.quantum_info.analysis.distance import hellinger_distance
from qiskit import QuantumRegister, ClassicalRegister, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
#from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from scipy.spatial import distance
from scipy.linalg import norm
from scipy.spatial.distance import euclidean
from qiskit import IBMQ
from qiskit.providers.aer.noise import NoiseModel
import matplotlib.pyplot as plt
import pandas as pd
# Loading your IBM Quantum account(s)
token = IBMQ.save_account('9d9d67f30e979c94f79826bb53a391c1b0a822660c6f49755656e8afde78092d159903b13b96ae0bd97b92dd3b8dc4f4c58168280ac0eb516673b46d781ec8a9',
                         overwrite=True)

In [2]:
def generate_plot(ideal_counts, calculated_counts, backend_name, circ_name):
    ideal_counts = dict(sorted(ideal_counts.items()))
    calculated_counts = dict(sorted(calculated_counts.items()))
    
    #Probability Distribution
    s_ideal = sum(ideal_counts.values())
    for key, value in ideal_counts.items():
        percentage = value / s_ideal
        ideal_counts[key] = percentage
        
    s_calculated = sum(calculated_counts.values())
    for key, value in calculated_counts.items():
        percentage = value / s_calculated
        calculated_counts[key] = percentage
    
    fname = f'./Images/Comparison_with_real/{backend_name}_{circ_name}.pdf'
        
    X = np.arange(len(calculated_counts))
    f, ax = plt.subplots(figsize=(12,8))
    ax.bar(X-0.2, calculated_counts.values(), width=0.4, align='center', edgecolor='black')
    ax.bar(X+0.2, ideal_counts.values(), width=0.4, align='center', edgecolor='black')
    ax.legend(('Simulated noisy probability','Experimental Probability'), fontsize=15)
    plt.xticks(X, calculated_counts.keys(), fontsize=13)
    plt.xticks(X, ideal_counts.keys(), fontsize=13)
    plt.yticks(fontsize=13)
    plt.xticks(rotation=90)
    plt.xlabel("States", fontsize=16)
    plt.ylabel("Probability", fontsize=16)
    plt.savefig(fname)
    plt.show()

In [3]:
distance_dataframe = pd.DataFrame(columns=["Backend", "State Name", "TVD Distribution", 
                                           "Jensen-Shannon Distance", "Hellinger Distance", "Counts", "Counts All Error"])

distance_dataframe

,Backend,State Name,TVD Distribution,Jensen-Shannon Distance,Hellinger Distance,Counts,Counts All Error


In [4]:
def save_to_csv():
    global distance_dataframe
    
    prev_df = pd.read_csv('./all_results.csv')
    new_df = pd.concat([prev_df, distance_dataframe])
    with open("./all_results.csv", "w") as f:
        new_df.to_csv(f, index=False)
        
    del distance_dataframe
    distance_dataframe = pd.DataFrame(columns=["Backend", "State Name", "TVD Distribution", 
                                           "Jensen-Shannon Distance", "Hellinger Distance", "Counts", "Counts All Error"])

In [5]:
def all_error(circ, qubits, circ_name):  
    global distance_dataframe
    
    reset_error = 0.05 # this acts on single qubit gate, probability of flip a singe qubit P(reset)
    measure_error = 0.2 # P(measure error)
    gate_error = 0.05 # P(two qubit error)

    # calling error functions using from IBM library
    re_err = pauli_error([('X', reset_error),('I',1-reset_error)])
    meas_err = pauli_error([('X', measure_error),('I', 1-measure_error)])
    gate_err1 = pauli_error([('X', gate_error),('I', 1-gate_error)])
    gate_err2 = gate_err1.tensor(gate_err1)

    # errors on a noisy model
    all_err = NoiseModel()
    all_err.add_all_qubit_quantum_error(re_err,"reset")
    all_err.add_all_qubit_quantum_error(meas_err,"measure")
    all_err.add_all_qubit_quantum_error(gate_err1,["u1", "u2", "u3"])
    all_err.add_all_qubit_quantum_error(gate_err2, "cx")
    
    # T1 and T2 values for qubits 0-3
    T1s = np.random.normal(50e3, 10e3, qubits) # Sampled from normal distribution mean 50 microsec
    T2s = np.random.normal(70e3, 10e3, qubits)  # Sampled from normal distribution mean 50 microsec

    # Truncate random T2s <= T1s
    T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(qubits)])

    # Instruction times (in nanoseconds)
    time_u1 = 0   # virtual gate
    time_u2 = 50  # (single X90 pulse)
    time_u3 = 100 # (two X90 pulses)
    time_cx = 300
    time_reset = 1000  # 1 microsecond
    time_measure = 1000 # 1 microsecond

    # QuantumError objects
    errors_reset = [thermal_relaxation_error(t1, t2, time_reset)
                for t1, t2 in zip(T1s, T2s)]
    errors_measure = [thermal_relaxation_error(t1, t2, time_measure)
                  for t1, t2 in zip(T1s, T2s)]
    errors_u1  = [thermal_relaxation_error(t1, t2, time_u1)
              for t1, t2 in zip(T1s, T2s)]
    errors_u2  = [thermal_relaxation_error(t1, t2, time_u2)
              for t1, t2 in zip(T1s, T2s)]
    errors_u3  = [thermal_relaxation_error(t1, t2, time_u3)
              for t1, t2 in zip(T1s, T2s)]
    errors_cx = [[thermal_relaxation_error(t1a, t2a, time_cx).expand(
             thermal_relaxation_error(t1b, t2b, time_cx))
              for t1a, t2a in zip(T1s, T2s)]
               for t1b, t2b in zip(T1s, T2s)]

    # Add errors to noise model
    noise_thermal = NoiseModel()
    for j in range(qubits):
        all_err.add_quantum_error(errors_reset[j], "reset", [j])
        all_err.add_quantum_error(errors_measure[j], "measure", [j])
        all_err.add_quantum_error(errors_u1[j], "u1", [j])
        all_err.add_quantum_error(errors_u2[j], "u2", [j])
        all_err.add_quantum_error(errors_u3[j], "u3", [j])
        for k in range(3):
            all_err.add_quantum_error(errors_cx[j][k], "cx", [j, k])
        
    
    
    p_gate = 0.1
    error_meas = pauli_error([('X',reset_error), ('I', 1 - reset_error)])
    error_gate1 = depolarizing_error(p_gate, qubits)
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_depolar = NoiseModel()
    all_err.add_all_qubit_quantum_error(error_meas, "measure", qubits)
    
    
    provider = IBMQ.load_account()
    get_provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
    backends = provider.backends()
    print(backends)
    backend_names = ['ibmq_lima', 'ibmq_belem', 'ibmq_quito', 'ibmq_manila', 'ibm_nairobi', 'ibm_oslo']
    
    for name in backend_names:
        if name == 'ibm_oslo':
            continue
        
        backend = provider.get_backend(name)
        if(backend.configuration().n_qubits > 2):
            print(backend, end='\n')
            
            noise_model = NoiseModel.from_backend(backend)
            print(noise_model, end='\n')
            
            coupling_map = backend.configuration().coupling_map
            
            basis_gates = noise_model.basis_gates
            print(basis_gates, end='\n')
            
            noise_model = NoiseModel.from_backend(backend).to_dict()
            real_run = execute(circ, backend= provider.get_backend(name), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model).result()
            
            counts = real_run.get_counts()
            plot_bit = plot_histogram(counts)
            display(plot_bit)
            sim_noise = AerSimulator(noise_model=all_err)
    
            tnoise = transpile(circ, sim_noise)
            result_all_err = sim_noise.run(tnoise).result()
            counts_all_err = result_all_err.get_counts()
    
            num_binary_states = 2**qubits

            # get the total counts for two dictionaries
            N1 = sum(counts.values())
            N2 = sum(counts_all_err.values())

            # add missing binary states in both distributions
            # example, dist1 = {'0': 400} and dist2 = {'1': 600}
            # the follwoing for loop will make them both same size (i.e., with same keys)
            # new dist1 = {'0': 400, '1': 0} and dist2 = {'0': 0, '1': 600}
            for number in range(2**qubits):
                # following line converts an integer to a binary string 
                # the binary string length is fixed and it is number of qubits
                # example, if number of qubit is 4, integer 3 will be `0011`
                binary_state = '{0:b}'.format(number).zfill(qubits)

                if binary_state not in counts:
                    counts[binary_state] = 0
        
                if binary_state not in counts_all_err:
                    counts_all_err[binary_state] = 0
                    
            generate_plot(counts, counts_all_err, name, circ_name)

            # following loop actually computes the TVD between two distributions
            tvd = 0
            print(' TVD Distribution')
            for key in counts:
                print(key)
                print(counts[key], counts_all_err[key])
                tvd = tvd + 0.5 * abs(counts[key]/N1 - counts_all_err[key]/N2)
                print(tvd)
    
    
            print('Jensen-Shannon Distance')
            a = counts.values()
            b = counts_all_err.values()
            data_a = list(a)
            data_b = list(b)
            arr_a = np.array(data_a)
            arr_b = np.array(data_b)
            jsd = distance.jensenshannon(arr_a, arr_b)
            print(jsd)
    
            print('Hellinger Distance') 
            hell = hellinger_distance(counts, counts_all_err)
            print(hell)
            
            distance_dataframe.loc[len(distance_dataframe.index)] = [name, circ_name, tvd, jsd, hell, counts, counts_all_err]
            
            save_to_csv()

In [6]:
#phase code
phase_q = QuantumRegister(5)
phase_c = ClassicalRegister(5)
phase_circ = QuantumCircuit(phase_q, phase_c)
phase_circ.h(phase_q[0])
phase_circ.initialize([1,0], 1)
phase_circ.x(phase_q[2])
phase_circ.initialize([1,0], 3)
phase_circ.h(phase_q[4])
phase_circ.h(phase_q[0:5])
phase_circ.cz(phase_q[0], phase_q[1])
phase_circ.h(phase_q[2])
phase_circ.h(phase_q[0])
phase_circ.cz(phase_q[1], phase_q[2])
phase_circ.h(phase_q[1])
phase_circ.cz(phase_q[2], phase_q[3])
phase_circ.h(phase_q[2])
phase_circ.cz(phase_q[3], phase_q[4])
phase_circ.h(phase_q[3])
phase_circ.h(phase_q[4])
phase_circ.barrier()
phase_circ.measure(phase_q[0:4], phase_c[0:4])
display(phase_circ.draw())


┌───┐       ┌───┐     ┌───┐                ░ ┌─┐         
q0_0: ───────┤ H ├───────┤ H ├──■──┤ H ├────────────────░─┤M├─────────
      ┌──────┴───┴──────┐├───┤  │  └───┘┌───┐           ░ └╥┘┌─┐      
q0_1: ┤ Initialize(1,0) ├┤ H ├──■────■──┤ H ├───────────░──╫─┤M├──────
      └──────┬───┬──────┘├───┤┌───┐  │  └───┘┌───┐      ░  ║ └╥┘┌─┐   
q0_2: ───────┤ X ├───────┤ H ├┤ H ├──■────■──┤ H ├──────░──╫──╫─┤M├───
      ┌──────┴───┴──────┐├───┤└───┘       │  └───┘┌───┐ ░  ║  ║ └╥┘┌─┐
q0_3: ┤ Initialize(1,0) ├┤ H ├────────────■────■──┤ H ├─░──╫──╫──╫─┤M├
      └──────┬───┬──────┘├───┤                 │  ├───┤ ░  ║  ║  ║ └╥┘
q0_4: ───────┤ H ├───────┤ H ├─────────────────■──┤ H ├─░──╫──╫──╫──╫─
             └───┘       └───┘                    └───┘ ░  ║  ║  ║  ║ 
c0: 5/═════════════════════════════════════════════════════╩══╩══╩══╩═
                                                           0  1  2  3

In [ ]:
all_error(phase_circ, len(phase_q), 'phase_circ')

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_manila') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_nairobi') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_oslo') from IBMQ(hub='ibm-q', group='open', project='main')>]
ibmq_lima


/home/protik/anaconda3/envs/qmlenv/lib/python3.9/site-packages/qiskit/providers/aer/noise/device/models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (213871.45710830402 > 2 * 76326.72328645267). Truncating to maximum value.
  warn("Device model returned an invalid T_2 relaxation time greater than"
/home/protik/anaconda3/envs/qmlenv/lib/python3.9/site-packages/qiskit/providers/aer/noise/device/models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (213871.45710830402 > 2 * 76326.72328645267). Truncating to maximum value.
  warn("Device model returned an invalid T_2 relaxation time greater than"
/home/protik/anaconda3/envs/qmlenv/lib/python3.9/site-packages/qiskit/providers/aer/noise/device/models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (213871.45710

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['cx', 'x', 'measure', 'reset', 'id', 'sx']
  Qubits with noise: [0, 1, 2, 3, 4]
  Specific qubit errors: [('id', (0,)), ('id', (1,)), ('id', (2,)), ('id', (3,)), ('id', (4,)), ('sx', (0,)), ('sx', (1,)), ('sx', (2,)), ('sx', (3,)), ('sx', (4,)), ('x', (0,)), ('x', (1,)), ('x', (2,)), ('x', (3,)), ('x', (4,)), ('cx', (4, 3)), ('cx', (3, 4)), ('cx', (0, 1)), ('cx', (1, 0)), ('cx', (3, 1)), ('cx', (1, 3)), ('cx', (2, 1)), ('cx', (1, 2)), ('reset', (0,)), ('reset', (1,)), ('reset', (2,)), ('reset', (3,)), ('reset', (4,)), ('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,))]
['cx', 'id', 'reset', 'rz', 'sx', 'x']


/home/protik/anaconda3/envs/qmlenv/lib/python3.9/site-packages/qiskit/execute_function.py:401: UserWarning: noise_model is not a recognized runtime option and may be ignored by the backend.
  job = backend.run(experiments, **run_kwargs)


In [ ]:
#bit code
bit_q  = QuantumRegister(5)
bit_c = ClassicalRegister(5)
bit_circ = QuantumCircuit(bit_q,bit_c)
bit_circ.barrier()
bit_circ.initialize([1,0], 1)
bit_circ.x(bit_q[2])
bit_circ.initialize([1,0], 3)
bit_circ.barrier()

bit_circ.cx(bit_q[0], bit_q[1])
bit_circ.cx(bit_q[2], bit_q[1])
bit_circ.cx(bit_q[2], bit_q[3])
bit_circ.cx(bit_q[4], bit_q[3])
bit_circ.barrier()
bit_circ.measure(bit_q[0:5],bit_c[0:5])
display(bit_circ.draw())


In [ ]:
all_error(bit_circ, len(bit_q), 'bit_circ')

In [ ]:
swap_qaoa_q = QuantumRegister(4)
swap_qaoa_c = ClassicalRegister(4)
swap_qaoa_circ = QuantumCircuit(swap_qaoa_q, swap_qaoa_c)
swap_qaoa_circ.h(swap_qaoa_q[0:4])
swap_qaoa_circ.cx(swap_qaoa_q[0], swap_qaoa_q[1])
swap_qaoa_circ.cx(swap_qaoa_q[2], swap_qaoa_q[3])
swap_qaoa_circ.rz(np.pi/2, 1)
swap_qaoa_circ.rz(np.pi/2, 2)
swap_qaoa_circ.cx(swap_qaoa_q[1], swap_qaoa_q[0])
swap_qaoa_circ.cx(swap_qaoa_q[3], swap_qaoa_q[2])
swap_qaoa_circ.cx(swap_qaoa_q[0], swap_qaoa_q[1])
swap_qaoa_circ.cx(swap_qaoa_q[2], swap_qaoa_q[3])
swap_qaoa_circ.cx(swap_qaoa_q[1], swap_qaoa_q[2])
swap_qaoa_circ.rz(np.pi/2, 3)
swap_qaoa_circ.cx(swap_qaoa_q[2], swap_qaoa_q[1])
swap_qaoa_circ.cx(swap_qaoa_q[1], swap_qaoa_q[2])
swap_qaoa_circ.cx(swap_qaoa_q[0], swap_qaoa_q[1])
swap_qaoa_circ.cx(swap_qaoa_q[2], swap_qaoa_q[3])
swap_qaoa_circ.rz(np.pi/2, 1)
swap_qaoa_circ.rz(np.pi/2, 3)
swap_qaoa_circ.cx(swap_qaoa_q[1], swap_qaoa_q[0])
swap_qaoa_circ.cx(swap_qaoa_q[3], swap_qaoa_q[2])
swap_qaoa_circ.cx(swap_qaoa_q[0], swap_qaoa_q[1])
swap_qaoa_circ.cx(swap_qaoa_q[2], swap_qaoa_q[3])
swap_qaoa_circ.cx(swap_qaoa_q[1], swap_qaoa_q[2])
swap_qaoa_circ.rz(np.pi/2, 2)
swap_qaoa_circ.cx(swap_qaoa_q[2], swap_qaoa_q[1])
swap_qaoa_circ.cx(swap_qaoa_q[1], swap_qaoa_q[2])
swap_qaoa_circ.rz(np.pi/3, 0)
swap_qaoa_circ.rz(np.pi/3, 1)
swap_qaoa_circ.rz(np.pi/3, 2)
swap_qaoa_circ.rz(np.pi/3, 3)
swap_qaoa_circ.barrier()
swap_qaoa_circ.measure(swap_qaoa_q[0:4], swap_qaoa_c[0:4])
display(swap_qaoa_circ.draw())

In [ ]:
all_error(swap_qaoa_circ, len(swap_qaoa_q), 'swap_qaoa_circ')

In [ ]:
vanilla_qaoa_q = QuantumRegister(3)
vanilla_qaoa_c = ClassicalRegister(3)
vanilla_qaoa_circ = QuantumCircuit(vanilla_qaoa_q, vanilla_qaoa_c)
vanilla_qaoa_circ.h(vanilla_qaoa_q[0:3])
vanilla_qaoa_circ.cx(vanilla_qaoa_q[0], vanilla_qaoa_q[1])
vanilla_qaoa_circ.rz(np.pi/2, 1)
vanilla_qaoa_circ.cx(vanilla_qaoa_q[0], vanilla_qaoa_q[1])
vanilla_qaoa_circ.cx(vanilla_qaoa_q[0], vanilla_qaoa_q[2])
vanilla_qaoa_circ.rz(np.pi/2, 2)
vanilla_qaoa_circ.cx(vanilla_qaoa_q[0], vanilla_qaoa_q[2])
vanilla_qaoa_circ.cx(vanilla_qaoa_q[1], vanilla_qaoa_q[2])
vanilla_qaoa_circ.rz(np.pi/2, 2)
vanilla_qaoa_circ.cx(vanilla_qaoa_q[1], vanilla_qaoa_q[2])
vanilla_qaoa_circ.rz(np.pi/3, 0)
vanilla_qaoa_circ.rz(np.pi/3, 1)
vanilla_qaoa_circ.rz(np.pi/3, 2)
vanilla_qaoa_circ.barrier()
vanilla_qaoa_circ.measure(vanilla_qaoa_q[0:3], vanilla_qaoa_c[0:3])
display(vanilla_qaoa_circ.draw())

In [ ]:
all_error(vanilla_qaoa_circ, len(vanilla_qaoa_q), 'vanilla_qaoa_circ')

In [ ]:
hamilton_q = QuantumRegister(3)
hamilton_c = ClassicalRegister(3)
hamilton_circ = QuantumCircuit(hamilton_q, hamilton_c)
hamilton_circ.h(hamilton_q[0:4])
hamilton_circ.rz(np.pi/2, hamilton_q[0:4])
hamilton_circ.h(hamilton_q[0:4])
hamilton_circ.cx(hamilton_q[0], hamilton_q[1])
hamilton_circ.rz(np.pi/3, 1)
hamilton_circ.cx(hamilton_q[0], hamilton_q[1])
hamilton_circ.cx(hamilton_q[1], hamilton_q[2])
hamilton_circ.rz(np.pi/3, 2)
hamilton_circ.cx(hamilton_q[1], hamilton_q[2])
hamilton_circ.barrier()
hamilton_circ.measure(hamilton_q[0:3], hamilton_c[0:3])
display(hamilton_circ.draw())

In [ ]:
all_error(hamilton_circ, len(hamilton_q), 'hamilton_circ')

In [ ]:
q_ghz = QuantumRegister(3)
c_ghz = ClassicalRegister(3)
ghz_circ = QuantumCircuit(q_ghz, c_ghz)
ghz_circ.h(q_ghz[0])
ghz_circ.cx(q_ghz[0], q_ghz[1])
ghz_circ.cx(q_ghz[1], q_ghz[2])
ghz_circ.measure(q_ghz[0:3], c_ghz[0:3])
ghz_circ.draw()

In [ ]:
all_error(ghz_circ, len(q_ghz), 'ghz_circ')

In [ ]:
mer_bell = QuantumRegister(3)
mer_bell_c =  ClassicalRegister(3)
mer_bell_circ = QuantumCircuit(mer_bell, mer_bell_c)
mer_bell_circ.rx(-pi/2, mer_bell[0])
mer_bell_circ.cx(mer_bell[0], mer_bell[1])
mer_bell_circ.cx(mer_bell[1], mer_bell[2])
mer_bell_circ.h(mer_bell[1])
mer_bell_circ.h(mer_bell[2])
mer_bell_circ.cx(mer_bell[0], mer_bell[2])
mer_bell_circ.cx(mer_bell[1], mer_bell[2])
mer_bell_circ.cx(mer_bell[2], mer_bell[0])
mer_bell_circ.cx(mer_bell[1], mer_bell[0])
mer_bell_circ.s(mer_bell[2])
mer_bell_circ.s(mer_bell[0])
mer_bell_circ.h(mer_bell[2])
mer_bell_circ.cz(mer_bell[0], mer_bell[1])
mer_bell_circ.h(mer_bell[0])
mer_bell_circ.s(mer_bell[1])
mer_bell_circ.h(mer_bell[1])
mer_bell_circ.measure(mer_bell[0:3], mer_bell_c[0:3])
display(mer_bell_circ.draw())

In [ ]:
all_error(mer_bell_circ, len(mer_bell), 'mermin_circ')